In [1]:
from Bio.Seq import Seq
from random import seed
from random import random

In [2]:
def process(lines=None):
#line processor for fastq parser
    ks = ['name', 'sequence', 'optional', 'quality']
    return {k: v for k, v in zip(ks, lines)}

In [3]:
def parseq(fn):
#open a fastq file and parse each set of four lines into a dict in a list
    n = 4
    fq = []
    with open(fn, 'r') as fh:
        lines = []
        for line in fh:
            lines.append(line.rstrip()) #remove \n characters
            if len(lines) == n:
                record = process(lines)
                fq.append(record)
                lines = []
    print(len(fq), 'lines loaded')
    return fq

In [4]:
def samProcess(line):
#line processor for SAM parser
    ks = ['QNAME','FLAG','RNAME','POS','MAPQ','CIGAR','RNEXT','PNEXT','TLEN','SEQ','QUAL']
    return {k: v for k, v in zip(ks, line.split('\t'))}

In [5]:
def parseSam(fn):
#open a SAM file and parse each line as a dict in a list
    sam = []
    with open (fn,'r') as samfile:
        for line in samfile:
            sam.append(samProcess(line.rstrip()))
    print (len(sam), 'lines loaded')
    return sam

In [6]:
def writeq(R, fn):
#write a fastq file    
    with open (fn,'w') as fh:
        for line in R:
            for k,v in line.items():
                fh.write(v + '\n')
            
    print(len(R), 'lines written')

In [7]:
R1 = parseq('Eye_test_R1.fastqsanger')

99404 lines loaded


In [8]:
R2 = parseq('Eye_test_R2.fastqsanger')

99404 lines loaded


In [9]:
sam = parseSam('SCN_snp.sam')

130 lines loaded


In [10]:
targetPos = 38592534
newBase = 'T'

In [11]:
seed(1)
changed = 0
het = True #if true half reads are changed i.e. heterozygous. If false all are changed - homozygous
for line in sam:
    reverse=False
    seqTargetBase = targetPos - int(line['POS']) #this is the position of the target base from the read start position POS
    if seqTargetBase <= 0: continue  #dont process lines which dont cover the target position
    print(line['QNAME'], line['POS'])
    print (line['SEQ'])
    value = random()
    if value > 0.5 and het: continue #use a random number to change only half of bases if het is True
    changed += 1 #count how many are changed
    flagBin = bin(int(line['FLAG'])) #get the FLAG number into binary string format
    if flagBin[-5] == '1':  #fifth bit in FLAG indicates if reverse complemented 1 or not 0
        reverse = True
        print ('reverse')
    else: print('forward')
    if flagBin[-7] =='1':  #seventh bit in FLAG indicates if this is the first read in the pair - its in R1
        print('R1')
        for i in range(len(R1)):
            if line['QNAME'] in R1[i]['name']:  #find the read in R1 fastq file
                print(R1[i]['name'])
                print ('index = ',i, ' targetPos = ', seqTargetBase)
                if reverse:
                    print(Seq(R1[i]['sequence']))
                    print(R1[i]['sequence'][-seqTargetBase-1])
                    print(R1[i]['sequence'][:-seqTargetBase-1] + str(Seq(newBase).complement()) + R1[i]['sequence'][-seqTargetBase:])
                    R1[i]['sequence'] = R1[i]['sequence'][:-seqTargetBase-1] + str(Seq(newBase).complement()) + R1[i]['sequence'][-seqTargetBase:]
                else:
                    print(R1[i]['sequence'])
                    print(R1[i]['sequence'][seqTargetBase])
                    print(R1[i]['sequence'][:seqTargetBase] + newBase + R1[i]['sequence'][seqTargetBase+1:])
                    R1[i]['sequence'] = R1[i]['sequence'][:seqTargetBase] + newBase + R1[i]['sequence'][seqTargetBase+1:]
    if flagBin[-8] =='1':  #eight bit indicates if this is the second read in the pair - its in R2
        print('R2')
        for i in range(len(R2)):
            if line['QNAME'] in R2[i]['name']:
                print(R2[i]['name'])
                print ('index = ',i, ' targetPos = ', seqTargetBase)
                if reverse:
                    print(Seq(R2[i]['sequence']))
                    print(R2[i]['sequence'][-seqTargetBase-1])
                    print(R2[i]['sequence'][:-seqTargetBase-1] + str(Seq(newBase).complement()) + R2[i]['sequence'][-seqTargetBase:])
                    R2[i]['sequence'] = R2[i]['sequence'][:-seqTargetBase-1] + str(Seq(newBase).complement()) + R2[i]['sequence'][-seqTargetBase:]
                else:
                    print(R2[i]['sequence'])
                    print(R2[i]['sequence'][seqTargetBase])
                    print(R2[i]['sequence'][:seqTargetBase] + newBase + R2[i]['sequence'][seqTargetBase+1:])
                    R2[i]['sequence'] = R2[i]['sequence'][:seqTargetBase] + newBase + R2[i]['sequence'][seqTargetBase+1:]
    print()
print(changed, ' reads out of ', len(sam), ' changed')

SRR3315784.3160695 38592434
TCAATAAACTGAGTGGCCTCTGGGTCAAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCAC
reverse
R1
@SRR3315784.3160695/1
index =  59132  targetPos =  100
GTGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTTCTATGAGATCTGGGAGAAATTTGACCCAGAGGCCACTCAGTTTATTGA
G
ATGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTTCTATGAGATCTGGGAGAAATTTGACCCAGAGGCCACTCAGTTTATTGA

SRR3315784.3160802 38592435
CAATAAACTGAGTGGCCTCTGGGTCAAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACG
SRR3315784.3160697 38592435
CAATAAACTGAGTGGCCTCTGGGTCAAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACG
SRR3315784.3160803 38592436
AATAAACTGAGTGGCCTCTGGGTCAAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGC
forward
R1
@SRR3315784.3160803/1
index =  59240  targetPos =  98
AATAAACTGAGTGGCCTCTGGGTCAAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGC
C
AATAAACTGAGTGGCCTCTGGGTCAAATTTCTCCCAGA


SRR3315784.3160711 38592455
GGGTCAAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGAT
reverse
R2
@SRR3315784.3160711/2
index =  59147  targetPos =  79
ATCATCCTGGAGAACTTCAGCGTGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTTCTATGAGATCTGGGAGAAATTTGACCC
G
ATCATCCTGGAGAACTTCAGCATGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTTCTATGAGATCTGGGAGAAATTTGACCC

SRR3315784.3160712 38592457
GTCAAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGATGG
reverse
R2
@SRR3315784.3160712/2
index =  59148  targetPos =  77
CCATCATCCTGGAGAACTTCAGCGTGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTTCTATGAGATCTGGGAGAAATTTGAC
G
CCATCATCCTGGAGAACTTCAGCATGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTTCTATGAGATCTGGGAGAAATTTGAC

SRR3315784.3160812 38592460
AAATTTCTCCCAGATCTCATAGAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAA
forward
R2
@SRR3315784.3160812/2
index =  59251  targetPos =  74



SRR3315784.3160766 38592481
GAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGA
SRR3315784.3160724 38592481
GAACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGA
SRR3315784.3160785 38592482
AACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAG
SRR3315784.3160725 38592482
AACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAG
reverse
R2
@SRR3315784.3160725/2
index =  59161  targetPos =  52
CTCATCGTGGTCAACATGTACATTGCCATCATCCTGGAGAACTTCAGCGTGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTT
G
CTCATCGTGGTCAACATGTACATTGCCATCATCCTGGAGAACTTCAGCATGGCCACGGAGGAGAGCACCGAGCCCCTGAGTGAGGACGACTTCGATATGTT

SRR3315784.3160726 38592482
AACATATCGAAGTCGTCCTCACTCAGGGGCTCGGTGCTCTCCTCCGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAG
reverse
R2
@SRR3315784.3160726/2
index =  59162  targetPos =  52
CTCATCGTGGTC


SRR3315784.3160786 38592526
CGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAGGAAGGAGATGATGATGTAGGTGGTGAAGAAGAGGATGCCCACGG
forward
R2
@SRR3315784.3160786/2
index =  59223  targetPos =  8
CGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAGGAAGGAGATGATGATGTAGGTGGTGAAGAAGAGGATGCCCACGG
C
CGTGGCCATGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAGGAAGGAGATGATGATGTAGGTGGTGAAGAAGAGGATGCCCACGG

SRR3315784.3160816 38592526
CGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAGGAAGGAGATGATGATGTAGGTGGTGAAGAAGAGGATGCCCACGG
forward
R2
@SRR3315784.3160816/2
index =  59252  targetPos =  8
CGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAGGAAGGAGATGATGATGTAGGTGGTGAAGAAGAGGATGCCCACGG
C
CGTGGCCATGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAGGAAGGAGATGATGATGTAGGTGGTGAAGAAGAGGATGCCCACGG

SRR3315784.3160819 38592526
CGTGGCCACGCTGAAGTTCTCCAGGATGATGGCAATGTACATGTTGACCACGATGAGGAAGGAGATGATGATGTAGGTGGTGAAGAAGAGGATGCCCACGG
forward
R2
@SRR3315784.3160819/2
index =  59255  targetPos =  8
CGT

In [ ]:
writeq(R1,'R1_test_1.fastq')

In [ ]:
writeq(R2,'R2_test_1.fastq')